# BatchNorm1d example
In this example we show the `BatchNorm1d` layer in action.

We create a random input tensor and apply the `BatchNorm1d` layer to it. We then show that the mean and standard deviation of the output tensor are close to zero and one respectively.
We then create another input tensor with a different mean and standard deviation and show that the output tensor has a mean and standard deviation close to zero and one respectively.

We then switch the `BatchNorm1d` layer to *evaluation* mode and show that the output tensor has a mean and standard deviation close to the mean and standard deviation of the input tensor.

Finally we show that the output tensor has a mean and standard deviation close to the mean and standard deviation of the input tensor when the `BatchNorm1d` layer is in *evaluation* mode.

In [1]:
import torch
import torch.nn as nn

In [2]:
# BatchNorm 1d demo
# 1d input random uniform
x1 = torch.rand(10, 5)*4 - 3
print(x1)

tensor([[-1.3053, -1.4607, -1.7705, -0.9231, -1.5066],
        [-0.0626,  0.6098, -0.9591,  0.4728, -0.6671],
        [-2.3981,  0.1995, -1.9241, -0.7997,  0.5044],
        [-1.9536, -2.8249,  0.0189, -2.0048,  0.3983],
        [-1.5816, -2.4541, -0.4603, -1.0412,  0.9465],
        [-0.4174,  0.6161, -1.1053, -0.0532, -1.5250],
        [-1.8714, -1.9446,  0.9274,  0.5816,  0.2945],
        [ 0.4576, -0.9657, -2.1261, -1.4612, -1.7325],
        [-0.6391, -2.0033,  0.9688, -2.8802, -2.3617],
        [-2.1025, -2.7825, -2.2888, -0.2244, -1.6572]])


In [3]:
x1.mean(), x1.std()

(tensor(-0.9849), tensor(1.1387))

In [4]:
# Create batch norm layer. The input has 5 features.
# The batch norm layer will normalize each feature.
# The batch norm layer will learn the mean and standard deviation of each feature.
bn = nn.BatchNorm1d(5, affine=True, momentum=0.1)

In [5]:
# Apply batch norm layer to input
y1 = bn(x1)
# Show mean and standard deviation of output
y1.mean(), y1.std()

(tensor(-1.4305e-08, grad_fn=<MeanBackward0>),
 tensor(1.0101, grad_fn=<StdBackward0>))

In [6]:
# Create another input tensor with a different mean and standard deviation
x2 = torch.rand(10, 5) * 3.0 + 10.0
# Show mean and standard deviation of input
x2.mean(), x2.std()

(tensor(11.6037), tensor(0.9162))

In [7]:
# Apply batch norm layer to input
y2 = bn(x2)
# and show mean and standard deviation of output
y2.mean(), y2.std()

(tensor(-2.2888e-07, grad_fn=<MeanBackward0>),
 tensor(1.0101, grad_fn=<StdBackward0>))

In [8]:
# Show the mode of the batch norm layer
bn.training

True

In [9]:
for p in bn.parameters():
	print(p)

Parameter containing:
tensor([1., 1., 1., 1., 1.], requires_grad=True)
Parameter containing:
tensor([0., 0., 0., 0., 0.], requires_grad=True)


In [10]:
# Show running mean and running variance of batch norm layer
bn.running_mean, bn.running_var

(tensor([1.0269, 1.0446, 1.1046, 1.0892, 1.0934]),
 tensor([0.9703, 1.0797, 1.0101, 1.0398, 1.0047]))

In [11]:
bn.num_batches_tracked

tensor(2)

In [12]:
bn.momentum

0.1

In [13]:
running_mean = 0.0
running_mean = bn.momentum * x1.mean(dim=0) + (1 - bn.momentum) * running_mean
running_mean = bn.momentum * x2.mean(dim=0) + (1 - bn.momentum) * running_mean
running_mean

tensor([1.0269, 1.0446, 1.1046, 1.0892, 1.0934])

# `BatchNorm1d` layer in *evaluation* mode

In [14]:
# Switch the batch norm layer to evaluation mode
bn.eval()


BatchNorm1d(5, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)

In [15]:
x3 = torch.rand(10, 5) * 8.0 + 20.0
x3.mean(), x3.std()

(tensor(23.8035), tensor(2.3658))

In [16]:
y3 = bn(x3)
y3.mean(), y3.std()

(tensor(22.5098, grad_fn=<MeanBackward0>),
 tensor(2.3973, grad_fn=<StdBackward0>))

In [17]:
# Let's check the manual calculation of the output
_y3 = (x3 - bn.running_mean)/torch.sqrt(bn.running_var + bn.eps) * bn.weight + bn.bias
_y3.mean(), _y3.std()

(tensor(22.5098, grad_fn=<MeanBackward0>),
 tensor(2.3973, grad_fn=<StdBackward0>))

In [18]:
torch.isclose(y3, _y3)

tensor([[True, True, True, True, True],
        [True, True, True, True, True],
        [True, True, True, True, True],
        [True, True, True, True, True],
        [True, True, True, True, True],
        [True, True, True, True, True],
        [True, True, True, True, True],
        [True, True, True, True, True],
        [True, True, True, True, True],
        [True, True, True, True, True]])